# 코랩 설정

아래 순서에 따라 코랩 환경을 설정합니다. 
1. 메뉴 > 런타임 > 런타임 유형 변경에서 "GPU"를 선택합니다.
2. 왼쪽 사이드바에서 파일을 선택한 뒤 data 폴더를 생성합니다.

# 데이터셋 다운로드

아래 순서에 따라 데이터셋 다운로드 및 코랩에 업로드를 수행합니다. 

1. https://aifactory.space/competition/data/2106 페이지에 접속합니다.
1. 하단에 qm9_train_data.pt 와 qm9_test_data.pt 파일을 클릭하여 다운로드 합니다. 
1. 왼쪽 사이드바 파일탭을 열어 "data" 폴더에 두 파일을 업로드 합니다.

# 베이스라인 코드 구동에 필요한 파일다운로드

In [10]:
!wget https://raw.githubusercontent.com/aifactory-team/AFCompetition/master/2106/gnn.yaml
!wget https://raw.githubusercontent.com/aifactory-team/AFCompetition/master/2106/qm9_dataset.py
!wget https://raw.githubusercontent.com/aifactory-team/AFCompetition/master/2106/model.py

--2022-09-18 16:53:47--  https://raw.githubusercontent.com/aifactory-team/AFCompetition/master/2106/gnn.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 449 [text/plain]
Saving to: ‘gnn.yaml’

gnn.yaml            100%[===================>]     449  --.-KB/s    in 0s      

2022-09-18 16:53:47 (29.7 MB/s) - ‘gnn.yaml’ saved [449/449]

--2022-09-18 16:53:48--  https://raw.githubusercontent.com/aifactory-team/AFCompetition/master/2106/qm9_dataset.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7680 (7.5K) [text/plain]
Saving t

# 베이스라인 구동에 필요한 패키지 설치

dgl 패키지를 설치하기 위해 현재 코랩의 파이썬 버전 및 cuda 버전을 확인합니다.

In [3]:
import sys
print(sys.version)

import torch

print("Torch version:{}".format(torch.__version__))
print("cuda version: {}".format(torch.version.cuda))
print("cudnn version:{}".format(torch.backends.cudnn.version()))

3.7.14 (default, Sep  8 2022, 00:06:44) 
[GCC 7.5.0]
Torch version:1.12.1+cu113
cuda version: 11.3
cudnn version:8302


https://www.dgl.ai/pages/start.html 에 접속해서 파이썬 및 코랩 버전에 해당하는 설치 명령을 복사합니다.

In [4]:
!pip install dgl-cu113 dglgo -f https://data.dgl.ai/wheels/repo.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.dgl.ai/wheels/repo.html
     |████████████████████████████████| 239.1 MB 17 kB/s 
     |████████████████████████████████| 63 kB 1.2 MB/s 
     |████████████████████████████████| 281 kB 24.7 MB/s 
     |████████████████████████████████| 36.8 MB 44 kB/s 
     |████████████████████████████████| 78 kB 9.8 MB/s 
     |████████████████████████████████| 45 kB 4.5 MB/s 
     |████████████████████████████████| 103 kB 73.2 MB/s 
     |████████████████████████████████| 109 kB 76.1 MB/s 
     |████████████████████████████████| 51 kB 868 kB/s 
     |████████████████████████████████| 41 kB 528 kB/s 
     |████████████████████████████████| 3.2 MB 61.9 MB/s 
     |████████████████████████████████| 546 kB 55.0 MB/s 
     |████████████████████████████████| 84 kB 4.6 MB/s 
     |████████████████████████████████| 121 kB 74.5 MB/s 
     |████████████████████████████████| 100 kB 

In [5]:
!pip install omegaconf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 79 kB 6.6 MB/s 
     |████████████████████████████████| 117 kB 44.0 MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144575 sha256=a6eb6a91089e9bfbe171eb3d9d75e385010c8bcd428be482dc7f7713181b2d9b
  Stored in directory: /root/.cache/pip/wheels/8b/8d/53/2af8772d9aec614e3fc65e53d4a993ad73c61daa8bbd85a873
Successfully built antlr4-python3-runtime


# 베이스라인 코드 실행 및 제출 결과 파일 생성

In [ ]:
import os
import torch
import numpy as np
import argparse
import torch.nn.functional as F
from torch import nn, optim
from torch.utils.data import DataLoader

from qm9_dataset import QM9DGLDataset
from omegaconf import OmegaConf
from model import GNN_model
import torch.nn as nn
from tqdm import tqdm

"""
    GPU Setup
"""
def gpu_setup(use_gpu, gpu_id):
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)

    if torch.cuda.is_available() and use_gpu:
        print('cuda available with GPU:',torch.cuda.get_device_name(0))
        device = torch.device("cuda")
    else:
        print('cuda not available')
        device = torch.device("cpu")
    return device


def to_np(x):
    return x.cpu().detach().numpy()


def train_epoch(epoch, model, loss_fnc, dataloader, optimizer, scheduler, FLAGS, device):
    model.train()
    num_iters = len(dataloader)
    for i, (g, y) in enumerate(dataloader):
        g = g.to(device)
        y = y.to(device)

        optimizer.zero_grad()

        # run model forward and compute loss
        pred = model(g)
        loss = loss_fnc(pred, y)

        # backprop
        loss.backward()
        optimizer.step()

        if i % FLAGS.train_params.print_epoch_interval == 0:
            print(f"[{epoch}|{i}] loss: {loss.item():.5f}")

        scheduler.step(epoch + i / num_iters)


def val_epoch(epoch, model, loss_fnc, dataloader, FLAGS, device):
    model.eval()
    total_loss = 0
    rescale_loss = 0
    for i, (g, y) in enumerate(tqdm(dataloader)):
        g = g.to(device)
        y = y.to(device)

        # run model forward and compute loss
        pred = model(g)
        loss = loss_fnc(pred, y)

        total_loss += loss.item()

    print(f"...[{epoch}|val] loss: {total_loss:.5f}")


def run_test(model, dataloader, device):
    model.eval()
    preds = []
    for g in tqdm(dataloader):
        g = g.to(device)
        pred = model(g)
        preds.append(to_np(pred))

    return np.concatenate(preds, axis=0)

# Loss function
def l1_loss(pred, target):
    loss = F.l1_loss(pred, target)
    return loss

################ 1. parsing arguments

parser = argparse.ArgumentParser()
parser.add_argument('--config', type=str, default='./gnn.yaml', help='configration for model')
parser.add_argument('--pretrained_path', type=str, default=None, help='configration for model')
args, extra_args = parser.parse_known_args()
FLAGS = OmegaConf.load(args.config)

# Create model directory
if not os.path.isdir(FLAGS.out_dir):
    os.makedirs(FLAGS.out_dir)

# Fix SEED
torch.manual_seed(FLAGS.train_params.seed)
np.random.seed(FLAGS.train_params.seed)

# Automatically choose GPU if available
device = gpu_setup(FLAGS['gpu']['use'], FLAGS['gpu']['id'])



################ 2. Prepare data
dataset = QM9DGLDataset(FLAGS.data_path,
                              FLAGS.task,
                              file_name='qm9_train_data.pt',
                              mode='train')

train_dataset, val_dataset = dataset.train_val_random_split(0.8)


train_loader = DataLoader(train_dataset,
                          batch_size=FLAGS.train_params.batch_size,
                          shuffle=True,
                          collate_fn=dataset.collate_fn,
                          num_workers=FLAGS.data.num_workers)

val_loader = DataLoader(val_dataset,
                        batch_size=FLAGS.train_params.batch_size,
                        shuffle=False,
                        collate_fn=dataset.collate_fn,
                        num_workers=FLAGS.data.num_workers)

# Test Dataset
test_dataset = QM9DGLDataset(FLAGS.data_path,
                         FLAGS.task,
                         file_name='qm9_test_data.pt',
                         mode='test')

test_loader = DataLoader(test_dataset,
                         batch_size=FLAGS.train_params.batch_size,
                         shuffle=False,
                         collate_fn=test_dataset.collate_fn,
                         num_workers=FLAGS.data.num_workers)

FLAGS.train_size = len(train_dataset)
FLAGS.val_size = len(val_dataset)
FLAGS.test_size = len(test_dataset)
print(f"Train set size: {len(train_dataset)}")
print(f"Validation set size: {len(val_dataset)}")
print(f"Test set size: {len(test_dataset)}")

################ 2. Prepare model
model = GNN_model(FLAGS.graph_encoder_params)

if args.pretrained_path is not None:
    model.load_state_dict(torch.load(args.pretrained_path))

model.to(device)

criterion = l1_loss
optimizer = optim.Adam(model.parameters(), lr=FLAGS.train_params.init_lr)
scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer,
                                                           FLAGS.train_params.epochs,
                                                           eta_min=FLAGS.train_params.min_lr)
                                                 

################ 2. Start training

# Run training
print('Begin training')
for epoch in range(FLAGS.train_params.epochs):
    train_epoch(epoch, model, criterion, train_loader, optimizer, scheduler, FLAGS, device)
    val_epoch(epoch, model, criterion, val_loader, FLAGS, device)
    
    # save checkpoint
    save_path = os.path.join(FLAGS.out_dir, f"{FLAGS.model}_{FLAGS.task}_{epoch}.pt")
    torch.save(model.state_dict(), save_path)
    print(f"Saved checkpoint: {save_path}")


################ 3. Test
print('Begin test')
predictions = run_test(model, test_loader, device)
np.savetxt('pred.csv', predictions)

cuda available with GPU: Tesla T4
Loaded train-set, task: mu, source: ./data/, length: 98123
Loaded test-set, task: mu, source: ./data/, length: 32708
Train set size: 78498
Validation set size: 19625
Test set size: 32708
Begin training
[0|0] loss: 2.65366
[0|5] loss: 2.18376
[0|10] loss: 1.65408
[0|15] loss: 1.22566
[0|20] loss: 1.45862
[0|25] loss: 1.19617
[0|30] loss: 1.13635
[0|35] loss: 1.32278
[0|40] loss: 1.21828
[0|45] loss: 1.06686
[0|50] loss: 1.16855
[0|55] loss: 1.20999
[0|60] loss: 1.13036
[0|65] loss: 1.15609
[0|70] loss: 1.04214
[0|75] loss: 1.05331
[0|80] loss: 0.93232
[0|85] loss: 1.09691
[0|90] loss: 1.08293
[0|95] loss: 1.05371
[0|100] loss: 1.08677
[0|105] loss: 1.06394
[0|110] loss: 1.07637
[0|115] loss: 1.05445
[0|120] loss: 1.03341
[0|125] loss: 1.14567
[0|130] loss: 1.14348
[0|135] loss: 0.97522
[0|140] loss: 1.17519
[0|145] loss: 0.98141
[0|150] loss: 1.01236
[0|155] loss: 0.89228
[0|160] loss: 1.06599
[0|165] loss: 1.06851
[0|170] loss: 1.01564
[0|175] loss: 1.

100%|██████████| 154/154 [00:08<00:00, 17.33it/s]


...[0|val] loss: 149.30917
Saved checkpoint: ./results/GNN_model_mu_0.pt
[1|0] loss: 0.92169
[1|5] loss: 0.96532
[1|10] loss: 0.92976
[1|15] loss: 0.95760
[1|20] loss: 1.06654
[1|25] loss: 0.97332
[1|30] loss: 0.91850
[1|35] loss: 0.88705
[1|40] loss: 1.10414
[1|45] loss: 1.03187
[1|50] loss: 0.95193
[1|55] loss: 0.73724
[1|60] loss: 1.05049
[1|65] loss: 1.04387
[1|70] loss: 0.89352
[1|75] loss: 0.85845
[1|80] loss: 1.05089
[1|85] loss: 0.90170
[1|90] loss: 0.97643
[1|95] loss: 1.02082
[1|100] loss: 1.01904
[1|105] loss: 0.99312
[1|110] loss: 0.84349
[1|115] loss: 0.89560
[1|120] loss: 1.02861
[1|125] loss: 0.88915
[1|130] loss: 1.06923
[1|135] loss: 1.01178
[1|140] loss: 0.94348
[1|145] loss: 1.02396
[1|150] loss: 0.85875
[1|155] loss: 0.89418
[1|160] loss: 0.96842
[1|165] loss: 1.12291
[1|170] loss: 0.92380
[1|175] loss: 0.97721
[1|180] loss: 1.07664
[1|185] loss: 1.08725
[1|190] loss: 0.94213
[1|195] loss: 0.93215
[1|200] loss: 1.10824
[1|205] loss: 0.94716
[1|210] loss: 0.96500
[1|

100%|██████████| 154/154 [00:11<00:00, 13.86it/s]


...[1|val] loss: 145.92267
Saved checkpoint: ./results/GNN_model_mu_1.pt
[2|0] loss: 0.93410
[2|5] loss: 0.95049
[2|10] loss: 0.88661
[2|15] loss: 0.95772
[2|20] loss: 0.87510
[2|25] loss: 1.04800
[2|30] loss: 1.10409
[2|35] loss: 0.89407
[2|40] loss: 0.93138
[2|45] loss: 0.95219
[2|50] loss: 0.99534
[2|55] loss: 1.02433
[2|60] loss: 0.91185
[2|65] loss: 1.05353
[2|70] loss: 0.83703
[2|75] loss: 0.99504
[2|80] loss: 0.93852
[2|85] loss: 0.87969
[2|90] loss: 1.02576
[2|95] loss: 0.98294
[2|100] loss: 0.90465
[2|105] loss: 0.95754
[2|110] loss: 0.95528
[2|115] loss: 1.12288
[2|120] loss: 0.87872
[2|125] loss: 0.94290
[2|130] loss: 1.05954
[2|135] loss: 0.91693
[2|140] loss: 1.06366
[2|145] loss: 0.98017
[2|150] loss: 0.94130
[2|155] loss: 0.89554
[2|160] loss: 1.04980
[2|165] loss: 0.87417
[2|170] loss: 0.88344
[2|175] loss: 0.94018
[2|180] loss: 0.97908
[2|185] loss: 1.07777
[2|190] loss: 1.01537
[2|195] loss: 0.95999
[2|200] loss: 0.98324
[2|205] loss: 0.95438
[2|210] loss: 1.12412
[2|

...

...[49|val] loss: 132.90775
Saved checkpoint: ./results/GNN_model_mu_49.pt
[50|0] loss: 0.87383
[50|5] loss: 0.76425
[50|10] loss: 0.77540
[50|15] loss: 0.95104
[50|20] loss: 0.85712
[50|25] loss: 0.89398
[50|30] loss: 0.96148
[50|35] loss: 0.88108
[50|40] loss: 0.77554
[50|45] loss: 0.93255
[50|50] loss: 0.94904
[50|55] loss: 0.85467
[50|60] loss: 0.84672
[50|65] loss: 0.84808
[50|70] loss: 0.88007
[50|75] loss: 1.03214
[50|80] loss: 0.75863
[50|85] loss: 0.97084
[50|90] loss: 0.83045
[50|95] loss: 0.80140
[50|100] loss: 0.85854
[50|105] loss: 0.82582
[50|110] loss: 0.92357
[50|115] loss: 0.85603
[50|120] loss: 0.82119
[50|125] loss: 0.89417
[50|130] loss: 0.91418
[50|135] loss: 0.91727
[50|140] loss: 0.93346
[50|145] loss: 0.88052
[50|150] loss: 0.91108
[50|155] loss: 0.87823
[50|160] loss: 0.78068
[50|165] loss: 0.87746
[50|170] loss: 0.96525
[50|175] loss: 0.72315
[50|180] loss: 0.99888
[50|185] loss: 0.87967
[50|190] loss: 0.76388
[50|195] loss: 0.80546
[50|200] loss: 0.84009
[50|

100%|██████████| 154/154 [00:08<00:00, 17.28it/s]


...[50|val] loss: 134.06899
Saved checkpoint: ./results/GNN_model_mu_50.pt
[51|0] loss: 0.90254
[51|5] loss: 0.95220
[51|10] loss: 0.75813
[51|15] loss: 0.88505
[51|20] loss: 0.82539
[51|25] loss: 0.79510
[51|30] loss: 0.96442
[51|35] loss: 0.74315
[51|40] loss: 0.80777
[51|45] loss: 0.84053
[51|50] loss: 0.81702
[51|55] loss: 0.78125
[51|60] loss: 0.90181
[51|65] loss: 0.89741
[51|70] loss: 0.92446
[51|75] loss: 0.83034
[51|80] loss: 0.86289
[51|85] loss: 0.90048
[51|90] loss: 0.88288
[51|95] loss: 0.94347
[51|100] loss: 0.86338
[51|105] loss: 0.84004
[51|110] loss: 0.71071
[51|115] loss: 0.90650
[51|120] loss: 0.85370
[51|125] loss: 0.90686
[51|130] loss: 0.76917
[51|135] loss: 0.94972
[51|140] loss: 0.97599
[51|145] loss: 0.90299
[51|150] loss: 0.80201
[51|155] loss: 0.79731
[51|160] loss: 0.82305
[51|165] loss: 0.98943
[51|170] loss: 0.82608
[51|175] loss: 0.88156
[51|180] loss: 0.75334
[51|185] loss: 0.79552
[51|190] loss: 0.86375
[51|195] loss: 0.97253
[51|200] loss: 0.81682
[51|

100%|██████████| 154/154 [00:08<00:00, 17.62it/s]


...[51|val] loss: 132.74537
Saved checkpoint: ./results/GNN_model_mu_51.pt
[52|0] loss: 0.81751
[52|5] loss: 0.92475
[52|10] loss: 0.79929
[52|15] loss: 0.98053
[52|20] loss: 0.83544
[52|25] loss: 0.89016
[52|30] loss: 0.89802
[52|35] loss: 0.78039
[52|40] loss: 0.72587
[52|45] loss: 0.93212
[52|50] loss: 0.81198
[52|55] loss: 0.95468
[52|60] loss: 0.83957
[52|65] loss: 0.76773
[52|70] loss: 0.91718
[52|75] loss: 1.01999
[52|80] loss: 0.86318
[52|85] loss: 0.85913
[52|90] loss: 0.81830
[52|95] loss: 0.89577
[52|100] loss: 0.73214
[52|105] loss: 0.84444
[52|110] loss: 0.82512
[52|115] loss: 0.95007
[52|120] loss: 0.92917
[52|125] loss: 1.18430
[52|130] loss: 0.86861
[52|135] loss: 1.03174
[52|140] loss: 0.89882
[52|145] loss: 0.81166
[52|150] loss: 0.79856
[52|155] loss: 0.95360
[52|160] loss: 0.84454
[52|165] loss: 0.88891
[52|170] loss: 0.90467
[52|175] loss: 0.88787
[52|180] loss: 0.98791
[52|185] loss: 0.79990
[52|190] loss: 0.79242
[52|195] loss: 0.84604
[52|200] loss: 0.94483
[52|

# 결과 파일 제출

1. 코랩 파일탭에서 생성된 "pred.csv"을 다운로드 합니다.
1. https://aifactory.space/competition/submission/2106 에 접속합니다.
1. "pred.csv"파일을 위 페이지에서 제출합니다.
1. https://aifactory.space/competition/leaderboard/2106 리더보드에서 자신의 점수 및 순위를 확인합니다.